# Shape Detection 
Idea: 
0) set environment 
1) get pointclouds
2) create persistence diagrams using ripserer 
3) create persistence silhuettes and concatenate them 
4) use kmeans clustering on persistence silhuettes 


## 0) Set environment 


In [ ]:
# activate packages set environment 
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.activate(".")
# include src files 
include("src/filtration.jl")
import .Filtration
include("src/preprocessing.jl")
import .Preprocessing
include("src/visualization.jl")
import .Visualization
include("src/persistence.jl")
import .Persistence
# add packages 
using Clustering
using StatsPlots
using LinearAlgebra

## 1) Get pointclouds 

In [1]:
# Which pointclouds 
shapes = ["circle", "sphere", "torus", "line_segment", "disk", "potato", "ellipsoid"]
path_changes = Dict("line_segment" => "line")
n_pershape = [1,12,23,24,29] 
n_datapoints = 100
good_layout = (3,2)

pointclouds = []
corr_results = []
for s in shapes 
    for i in n_pershape
        push!(corr_results, s*"_"*lpad(i,4,'0'))
        # load the data 
        if s in keys(path_changes) 
            path = "data\\"*s*"\\samples\\"*path_changes[s]*"_"*lpad(i,4,'0')*".csv"
        else 
            path = "data\\"*s*"\\samples\\"*s*"_"*lpad(i,4,'0')*".csv"
        end 
        println(path)
        df = Preprocessing.load_pointcloud_csv(path)
        if n_datapoints != "end"
            df = df[1:n_datapoints,:]
        end 
        points = [tuple(df[i, :]...) for i in axes(df, 1)]
        push!(pointclouds, points)
    end 
end 

data\circle\samples\circle_0001.csv


LoadError: UndefVarError: `Preprocessing` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
## visualize point cloud 
# idx = findfirst( ==("potato_0006"), corr_results)
# Visualization.plot_pointcloud_3d(pointclouds[8], check_npoints = false)

## 2) Persistence diagrams 

In [ ]:
diagrams = Persistence.create_persistence_diagram(pointclouds, "Ripserer", true)

## 3) Silhuette

In [ ]:
silhouettes = Persistence.create_persistence_silhoutte(diagrams, weights = (0.2, 0.6, 0.2))

## 4) Clustering

In [ ]:
# works great for circle, sphere, torus, line_segment, disk each first 3 pointclouds
X = [Vector{Float64}(s) for s in silhouettes]
X = hcat(X...)
for i in 1:size(X)[2]
    # normalize each col
    if norm(X[:,i]) != 0
        X[:,i] = X[:,i] ./ norm(X[:,i])
    end 
end 
res_clus_sil = kmeans(X, length(shapes))
clusters_sil = res_clus_sil.assignments
for cluster_idx in unique(clusters_sil)
    println("Cluster ",cluster_idx,"---------")
    println(corr_results[clusters_sil .== cluster_idx])
end